In [1]:
# Use

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas, tqdm_notebook


In [2]:
tqdm.pandas()

In [ ]:
!ls | grep bios


In [3]:

df = pd.read_csv('../data/raw/harvard_bios.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10806 entries, 0 to 10805
Data columns (total 2 columns):
Unnamed: 0    10806 non-null object
0             10774 non-null object
dtypes: object(2)
memory usage: 168.9+ KB


In [ ]:
df.head()

We'll need to break up the strings into blocks and only use the markup that we actually need.

In [ ]:
# the output needs to be handled 1 at a time, and then dumped out carefully.
# df['soup'] = df['0'].progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [ ]:
# tSoup = df['0'].head().progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [ ]:
# ts = tSoup.iloc[0]

In [4]:
import json

In [5]:
def tableToDict(soup):
    "Given a headerless table, return a dict with all the fields"
    table = {}
    for row in soup.find_all('tr'):
        cells = row.find_all('td')
        table[str(cells[0].text.strip())] = cells[1].text.strip()
        
    return table
    
def soupToDict(soup):
    "Given a player's soup blob, parse their header"
    playerData = tableToDict(soup.select('div.player-info')[0].find('table'))
    playerData['name'] = soup.select('div.player-name span.name')[0].text.strip()
    return playerData
def getSynopsis(soup):
    "Given a player's soup, split their bio information into \
    chunks for each strong header"
    
    # Get all top level children
#     bioPs = soup.select('div.synopsis')[0].find_all('p', recursive=False)
    
#     blocks= []
#     # if their blocks are malformed, skip that block.
#     for i, val in enumerate(bioPs):
#         try:
#             blocks[val.select('strong')[0].text] = val.get_text()
#         except:
#             printf("Text not detected")
        
    
    # Given the variation in blob types, just return
    # single blob for now.
    # Ryan Brenner: b instead of strong tags
    # Jackson Stallings (Junior Year) - UL blobs mixed with P tags
    # Jackson Stallings (Freshman) - Clean example
    
    # Sometimes, no bio
#     http://yalebulldogs.com/sports/c-sail/2016-17/bios/buehler_patrick_nu3o?view=news
    bio = soup.find('div',class_='synopsis')
    return bio.get_text() if bio else None

In [ ]:
# ts.select('div.synopsis')[0].find_all('b')

In [ ]:
# getSynopsis(ts)

In [ ]:
# soupToDict(ts)

In [ ]:
# df['types'] = df['0'].map(type)
# df['types'].describe()

In [ ]:
# df['types']

In [6]:
df['headers'] = df['0'].progress_apply(lambda x: \
                                       json.dumps(soupToDict(BeautifulSoup(x, 'lxml')), encoding='utf-8') if (type(x) == str) else None)

100%|█| 10806/10806 [03:04<00:00, 58.54it/s]


In [7]:
df['synopsis'] = df['headers'].progress_apply(lambda x: 0)

100%|█| 10806/10806 [00:00<00:00, 491180.16it/s]


In [ ]:
# df['synopsis'] = df['0'].progress_apply(lambda x: getSynopsis(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

In [10]:
df.columns = ['url', 'raw', 'headers', 'synopsis']

In [11]:
df.index = df['url']

In [ ]:
df.head()

In [12]:
df[['synopsis', 'headers']].to_csv('harvard_bio_parsed.csv', encoding='utf8')

In [ ]:
# !gzip -c harvard_bio_parsed.csv > harvard_bio_parsed.csv.gz

In [ ]:
# if we redo this, batching it in databricks / spark job will save a lot of time.

In [ ]:
!ls -lS | grep parsed